In [63]:
import pandas as pd 
import requests 
# %conda install lxml
from io import StringIO

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"
res = requests.get(url=url, headers=headers).text

df = pd.read_html(res)
df

C:\Users\jesse\AppData\Local\Temp\ipykernel_11324\2308053695.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res)


[    0                                                  1
 0 NaN  This section needs additional citations for ve...,
    Rank              Player     Club                  Country and other  \
    Rank              Player   League  Cup Continental Country and other   
 0     1   Cristiano Ronaldo   590[b]   57         172               143   
 1     2       Lionel Messi*   553[c]   71         157               115   
 2     3               Pelé*   604[d]   49          26                83   
 3     4             Romário   545[e]   93          54                64   
 4     5       Ferenc Puskás   516[f]   69          56                84   
 5     6        Josef Bican*   515[g]  137          38                32   
 6     7  Robert Lewandowski   423[h]   62         117                88   
 7     8        Jimmy Jones*   330[i]  286          14                 9   
 8     9        Gerd Müller*   405[j]   92          69                68   
 9    10       Joe Bambrick*   347[k]  253     

In [64]:
df = df[3]

In [65]:
for col in df.columns:
    print(f"{col}:{pd.api.types.is_numeric_dtype(col)}")

Rank:False
Player:False
Goals:False
Matches:False
Ratio:False
Career span:False


In [66]:
df["Career span"] 

0        1924–1951
1     2002–present
2        1930–1957
3        1929–1961
4     2003–present
          ...     
78       1968–1996
79       1891–1920
80       1933–1955
81       1966–1986
82       1952–1971
Name: Career span, Length: 83, dtype: object

In [67]:
df["Career span"]

0        1924–1951
1     2002–present
2        1930–1957
3        1929–1961
4     2003–present
          ...     
78       1968–1996
79       1891–1920
80       1933–1955
81       1966–1986
82       1952–1971
Name: Career span, Length: 83, dtype: object

In [68]:
df["Career span"] = df["Career span"].str.split("–").str[0].astype(int)

In [69]:
df["Matches"] = [int(item.replace("+", "")) for item in df["Matches"]]

In [70]:
for col in df.columns:
    print(f"{col} is numeric? {pd.api.types.is_numeric_dtype(df[col])}")

Rank is numeric? True
Player is numeric? False
Goals is numeric? False
Matches is numeric? True
Ratio is numeric? True
Career span is numeric? True


In [71]:
df["Goals"] = df["Goals"].str.replace("+", "").astype(int)
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989,582,1.70,1924
1,2,Cristiano Ronaldo,975,1337,0.73,2002
2,3,Josef Bican,950,624,1.52,1930
3,4,Ronnie Rooke,934,1030,0.91,1929
4,5,Lionel Messi,925,1194,0.77,2003


In [ ]:
from datetime import datetime, timedelta
import random, math
data = []
columns = ["name", "date", "goals"]

for row in df.itertuples(): 
    name = row.Player
    start_date = f"{row._6}-08-01"
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    sd = random.random()
    for i in range(row.Matches):
        name = name
        date = timedelta (days=3 * i) + start_date
        goals = random.normalvariate(row.Ratio, sd)
        goals = math.round(goals) if goals > 0 else 0
        info = [name, date, goals]
        data.append(info)
    
games_df = pd.DataFrame(data=data, columns = columns)
games_df.shape

(60168, 3)

In [78]:
games_df.loc[games_df["goals"] > 3]

,name,date,goals
28671,Franz Binder,1929-02-12,4
28748,Franz Binder,1929-10-01,4


In [81]:
games_df.loc[games_df["goals"] < 0]

,name,date,goals


In [ ]:
fake_df = games_df.groupby("name").agg(
    matches = ("name", "count"),
    goals = ("goals", "sum"), 
    ratio = ("goals", "mean"), 
    career_start = ("date", "min"), 
)